# Assignment 2: Initial Data Prep

In this initial phase of the assignment work, the csv file originally used for the first assignment was cleaned again using the same techniques as had been originally used in that same first assignment. There were some noted differences in this approach following the first assignment feedback.

- The address column was kept in for this data preparation. While it is unlikely to provide much additional benefit to the model itself given its high cardinality, it could help in filling in other data for other columns and so was kept in
- The price feature was not binned as it had been last time but instead was kept in its original state. Initial analysis suggests that clamping this feature might be a more appropriate means to deal with outliers. However it was decided based on the recommendation from Kelleher, Mac Namee and D'Arcy (2020) to test the effectiveness of trained models on test sets that have both been clamped and that haven't been in order to see the effect of clamping the outliers on the model performances. As such, these values will be clamped following creating the test and train sets in another notebook
- 2022 Sale values were not removed from the dataset. While they were underrepresented in this first instance, dummy values could potentially be created to allow for a more even sampling of data and trends in the data

Other than the changes listed here, other data cleaning and preparatory operations were kept the same. The original notebook will be attached with this one, containing a more in-depth analysis and justification for certain steps taken. This notebook is designed to be a much more brief cleaning and preparation of the data

## Initial Assessment of Data

In [2]:
# Import necessary modules from both own work and as seen in sample notebook (see reference to this in markdown above)

import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import matplotlib.patches as mpatches
from geopy.geocoders import Nominatim
from matplotlib.backends.backend_pdf import PdfPages

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#Read in initial csv

rppr_df = pd.read_csv('ppr-14332591.csv', keep_default_na = True, delimiter=',', skipinitialspace = True)

In [3]:
rppr_df.dtypes

Date of Sale (dd/mm/yyyy)    object
Address                      object
Postal Code                  object
County                       object
Price (€)                    object
Not Full Market Price        object
VAT Exclusive                object
Description of Property      object
Property Size Description    object
dtype: object

In [4]:
# Note that price is excluded from this first alteration as more work was done to clean that feature
# Each datatype was assigned individually

rppr_df['Date of Sale (dd/mm/yyyy)'] = rppr_df['Date of Sale (dd/mm/yyyy)'].astype('datetime64[ns]')
rppr_df['Address'] = rppr_df['Address'].astype('category')
rppr_df['Postal Code'] = rppr_df['Postal Code'].astype('category')
rppr_df['County'] = rppr_df['County'].astype('category')
rppr_df['Not Full Market Price'] = rppr_df['Not Full Market Price'].astype('category')
rppr_df['VAT Exclusive'] = rppr_df['VAT Exclusive'].astype('category')
rppr_df['Description of Property'] = rppr_df['Description of Property'].astype('category')
rppr_df['Property Size Description'] = rppr_df['Property Size Description'].astype('category')

In [5]:
# Check the reassigning of datatypes

rppr_df.dtypes

Date of Sale (dd/mm/yyyy)    datetime64[ns]
Address                            category
Postal Code                        category
County                             category
Price (€)                            object
Not Full Market Price              category
VAT Exclusive                      category
Description of Property            category
Property Size Description          category
dtype: object

In [6]:
rppr_df.columns

Index(['Date of Sale (dd/mm/yyyy)', 'Address', 'Postal Code', 'County',
       'Price (€)', 'Not Full Market Price', 'VAT Exclusive',
       'Description of Property', 'Property Size Description'],
      dtype='object')

In [8]:
# Alter current column titles to make them easier to use programmatically in Python

rppr_df.columns = ['Date', 'Address', 'Postal_Code', 'County', 'Price', 'Not_Full_Market_Price',
                  'VAT_Exclusive', 'Description_of_Property', 'Property_Size_Description']

In [9]:
# Remove non-numerical characters from the price feature

for i in range(len(rppr_df.Price)):
    rppr_df.Price[i] = rppr_df.Price[i].replace('€','')
    rppr_df.Price[i] = rppr_df.Price[i].replace(',', '')

In [10]:
# Check that adjustment to price feature display worked

rppr_df.Price.head(5)

0    260000.00
1     81000.00
2    155000.00
3     60000.00
4    277533.00
Name: Price, dtype: object

In [11]:
# Alter price feature datatype

rppr_df['Price'] = pd.to_numeric(rppr_df['Price'])

In [12]:
# Check changing datatype was successful

rppr_df.dtypes

Date                         datetime64[ns]
Address                            category
Postal_Code                        category
County                             category
Price                               float64
Not_Full_Market_Price              category
VAT_Exclusive                      category
Description_of_Property            category
Property_Size_Description          category
dtype: object

In [13]:
# Check for duplicate rows

rppr_df.duplicated().sum()

0

In [14]:
# Check for duplicate columns

rppr_df_T = rppr_df.T
rppr_df_T.duplicated().sum()

0

In [15]:
# Assess the categorical columns with initial descriptive stats to see their unique values i.e. cardinality

category_columns = rppr_df.select_dtypes(['category']).columns
rppr_df[category_columns].describe().T

,count,unique,top,freq
Address,10000,9984,"120 COLLEGE VIEW, BALLYMUN, DUBLIN 11",2
Postal_Code,1914,22,Dublin 15,249
County,10000,26,Dublin,3190
Not_Full_Market_Price,10000,2,No,9501
VAT_Exclusive,10000,2,No,8425
Description_of_Property,10000,2,Second-Hand Dwelling house /Apartment,8395
Property_Size_Description,975,4,greater than or equal to 38 sq metres and less...,696


In [16]:
# Check the cardinality of price, which should be high as a continuous feature

rppr_df['Price'].nunique()

2270

In [17]:
# Check initial descriptive statistics to also examine range and standard deviation, for further assessment
# on cardinality and the overall shape of the data

rppr_df['Price'].describe()

count    1.000000e+04
mean     2.534407e+05
std      3.196092e+05
min      5.179000e+03
25%      1.180000e+05
50%      2.000000e+05
75%      3.083700e+05
max      1.630837e+07
Name: Price, dtype: float64

In [18]:
# Check the cardinality of date which should be high as a continuous feature

rppr_df['Date'].nunique()

2768

In [19]:
# View descriptive statistics of date to get sense of shape of the data

rppr_df['Date'].describe()

count                   10000
unique                   2768
top       2013-12-20 00:00:00
freq                       34
first     2010-01-02 00:00:00
last      2022-12-01 00:00:00
Name: Date, dtype: object

In [20]:
# check for the presence of null values in the dataframe

rppr_df.isnull().sum()

Date                            0
Address                         0
Postal_Code                  8086
County                          0
Price                           0
Not_Full_Market_Price           0
VAT_Exclusive                   0
Description_of_Property         0
Property_Size_Description    9025
dtype: int64

## Geolocating Addresses

In [21]:
# Creating small random sample from the data using just the first 100 rows

first100 = pd.DataFrame(rppr_df.head(100))
first100

,Date,Address,Postal_Code,County,Price,Not_Full_Market_Price,VAT_Exclusive,Description_of_Property,Property_Size_Description
0,2016-01-03,"80 THE WATER RILL, WATERSIDE SWORDS RD, MALAHI...",NaN,Dublin,260000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
1,2013-03-15,"apt 40, 109 parnell street",Dublin 1,Dublin,81000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
2,2011-01-14,"Cornabrone, Ballinamore",NaN,Leitrim,155000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
3,2016-10-27,"46 TI CHORMAIC, GLEANN NA RI, MURROUGH AVE",NaN,Galway,60000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
4,2014-08-22,"29 Barrettstown Meadows, Roseberry, NEWBRIDGE",NaN,Kildare,277533.0,No,Yes,New Dwelling house /Apartment,greater than 125 sq metres
...,...,...,...,...,...,...,...,...,...
95,2017-10-02,"THOMAS ST, CASTLEBLAYNEY, MONAGHAN",NaN,Monaghan,75000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
96,2010-02-26,"5 Glendoher Park, Rathfarnham",NaN,Dublin,410000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
97,2015-02-12,"TURLOUGH, CASTLEBAR, CO MAYO",NaN,Mayo,68000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
98,2016-10-13,"19 NEWTOWN VILLAS, BLACKROCK, DUBLIN",NaN,Dublin,373207.0,No,No,Second-Hand Dwelling house /Apartment,NaN


In [31]:
# Create a list of updated address values, all modified to lower case and removing some values that 
# that may cause errors in the address look-up or potential inconsistencies within the data returned
# Addresses were also converted to lowercase to make future analysis and comparison easier

first100_addresses_list = []
first100_addresses = first100['Address']
for address in first100_addresses:
    lowercase_address = address.lower()
    split_up_address = lowercase_address.rsplit(",", 1)
    if len(split_up_address) > 1:
        split_up_address.remove(split_up_address[0])
    print(split_up_address)
    first100_addresses_list.append(split_up_address)

[' malahide dublin']
[' 109 parnell street']
[' ballinamore']
[' murrough ave']
[' newbridge']
[' hansfield']
[' moyvane']
[' cork']
[' ballyhaunis']
[' caherdavin']
[' galway']
[' roscommon']
[' county roscommon']
[' leopardstown']
[' smithfield dublin 7']
[' raheny dublin 5']
[' midleton']
[' galway']
[' dublin']
[' ardara']
[' clarinbridge']
[' malahide road']
[' galway']
[' sligo']
[' straffan']
[' killarney']
[' dublin']
[' lehenaghmore']
[' donegal']
[' finglas dublin 11']
[' enfield']
[' bandon']
[' birr']
[' piltown']
[' dublin 13']
[' greystones']
[' dundrum']
[' co. clare']
[' dublin 24']
[' dublin 15']
[' co. donegal']
[' dublin 3']
[' bantry']
[' waterford']
[' lifford']
[' leopardstown']
[' drogheda']
[' meath']
[' dunlaoire']
[' sandyford']
[' dublin 11']
[' rathfarnham']
[' south dock rd dublin 4']
[' tramore']
[' dublin 6']
[' waterford']
[' mallow']
[' johnswell rd']
[' wicklow']
[' knockboy']
[' skerries']
[' dublin 5']
[' tipperary']
[' louth']
[' limerick']
[' dubli

In [36]:
# Print out geocoded addresses to check that geocoding is working before commiting to running on all values

geolocator = Nominatim(user_agent = 'property_analysis')

for i in range(len(first100_addresses_list)):
    # print(address[0] + "," + address[1])
    address_string = first100_addresses_list[i][0]
    # print(address_string)
    try:
        location = geolocator.geocode(address_string + ", Ireland")
        print(location.address)
    except Exception:
        print(f"Geolocator couldn't get the address for address at position {i}")
    time.sleep(1)

Malahide, Fingal, County Dublin, Leinster, K36 HN84, Éire / Ireland
Parnell Street, Rotunda B ED, Dublin, Dublin 1, Leinster, D01 R243, Éire / Ireland
Ballinamore, Ballinamore Municipal District, County Leitrim, Connacht, Éire / Ireland
Murrough Avenue, Murrough, Cathair na Gaillimhe, County Galway, Connacht, Éire / Ireland
Newbridge, The Municipal District of Kildare — Newbridge, County Kildare, Leinster, W12 AD93, Éire / Ireland
Hansfield, Blanchardstown-Blakestown ED, Fingal, County Dublin, Leinster, Éire / Ireland
Moyvane, Listowel Municipal District, County Kerry, Munster, Éire / Ireland
Cork, County Cork, Munster, T12 T997, Éire / Ireland
Ballyhaunis, Claremorris-Swinford Municipal District, County Mayo, Connacht, F35 NF82, Éire / Ireland
Caherdavin, The Metropolitan District of Limerick City, Limerick, County Limerick, Munster, V94TN5P, Éire / Ireland
Cathair na Gaillimhe, County Galway, Connacht, Éire / Ireland
County Roscommon, Connacht, Éire / Ireland
County Roscommon, Connac

In [37]:
# Create a list of updated address values, all modified to lower case and removing some values that 
# that may cause errors in the address look-up or potential inconsistencies within the data returned
# Addresses were also converted to lowercase to make future analysis and comparison easier

addresses_list = []
addresses = rppr_df['Address']
for address in addresses:
    lowercase_address = address.lower()
    split_up_address = lowercase_address.rsplit(",", 1)
    if len(split_up_address) > 1:
        split_up_address.remove(split_up_address[0])
    print(split_up_address)
    addresses_list.append(split_up_address)

[' malahide dublin']
[' 109 parnell street']
[' ballinamore']
[' murrough ave']
[' newbridge']
[' hansfield']
[' moyvane']
[' cork']
[' ballyhaunis']
[' caherdavin']
[' galway']
[' roscommon']
[' county roscommon']
[' leopardstown']
[' smithfield dublin 7']
[' raheny dublin 5']
[' midleton']
[' galway']
[' dublin']
[' ardara']
[' clarinbridge']
[' malahide road']
[' galway']
[' sligo']
[' straffan']
[' killarney']
[' dublin']
[' lehenaghmore']
[' donegal']
[' finglas dublin 11']
[' enfield']
[' bandon']
[' birr']
[' piltown']
[' dublin 13']
[' greystones']
[' dundrum']
[' co. clare']
[' dublin 24']
[' dublin 15']
[' co. donegal']
[' dublin 3']
[' bantry']
[' waterford']
[' lifford']
[' leopardstown']
[' drogheda']
[' meath']
[' dunlaoire']
[' sandyford']
[' dublin 11']
[' rathfarnham']
[' south dock rd dublin 4']
[' tramore']
[' dublin 6']
[' waterford']
[' mallow']
[' johnswell rd']
[' wicklow']
[' knockboy']
[' skerries']
[' dublin 5']
[' tipperary']
[' louth']
[' limerick']
[' dubli

[' meath']
[' cork']
[' donegal']
[' naas']
[' tipperary']
[' clontarf dublin 3']
[' mayo']
[' firhouse']
[' fermoy']
[' golf links road']
[' co offaly']
[' sandyford']
[' tuam']
[' offaly']
[' maynooth']
[' dublin 18']
[' co waterford']
[' cortoon']
[' co donegal']
[' navan']
[' meath']
[' leeson park']
[' navan']
[' adamstown  lucan']
[' clonskeagh dublin 14']
[' dublin 4']
[' cork']
[' collinstown']
[' kilmallock']
[' kilrush']
[' wicklow']
[' dublin']
[' co dublin']
[' south circular road']
[' portlaoise']
[' dungarvan']
[' roscommon']
[' tullamore']
[' dublin 16']
[' killarney']
[' coolock dublin 17']
[' ashbourne']
[' co galway']
[' drogheda']
[' gaultier']
[' galway']
[' dalkey']
[' dublin']
[' waterford']
[' laois']
[' lissarda']
[' artane']
[' dublin 16']
[' dublin 1']
[' cabinteely dublin 18']
[' clare']
[' waterford']
[' rosbrien']
[' maryborough hill']
[' carlow']
[' youghal']
[' dublin 9']
[' dublin 4']
[' waterford']
[' murroe']
[' dublin 11']
[' longford']
[' dun laoghai

[' tuam']
[' caltragh']
[' bantry']
[' kill']
[' cork']
[' tipperary']
[' dublin 15']
[' dublin 5']
[' killurin']
[' rathmines dublin 6']
[' dublin 7']
[' ballyfermot']
[' louth']
[' delgany']
[' boyle']
[' westmeath']
[' piltown']
[' celbridge']
[' lucan']
[' ennis']
[' blackwater']
[' cruise park']
[' blackhall place']
[' galway']
[' donegal']
[' the quay']
[' wexford']
[' ranelagh']
[' tullow']
[' gorey']
[' limerick']
[' balbriggan']
[' kilmainham']
[' dundalk']
[' limerick']
[' ardara']
[' stepaside dublin 18']
[' kildare']
[' stradbally']
[' donegal']
[' dublin 6']
[' swinford']
[' passage east']
[' artane dublin 5']
[' donabate']
[' dublin 22']
[' galway city']
[' dublin']
[' wicklow']
[' navan road']
[' meath']
[' dublin']
[' burnfoot']
[' trevet']
[' dunsany']
[' cavan']
[' clane']
[' killure']
[' roundwood']
[' phibsborough']
[' dunboyne']
[' carrig']
[' marianella']
[' dublin 6w']
[' tuam']
[' dublin 6w']
[' dublin 6w']
[' kerry']
[' portarlington']
[' sandycove']
[' ennisco

[' kildare']
[' kilkenny']
[' kenmare']
[' athy']
[' kerry']
[' county kildare']
[' raheen']
[' bray rd']
[' clondalkin']
[' athboy']
[' clare']
[' killarney']
[' macroom']
[' cuffesgrange']
[' strandhill']
[' blackrock']
[' bundoran']
[' meath']
[' cork']
[' co cavan']
[' butterfield ave']
[' castlefin']
[' ballymacoda']
[' ashbourne']
[' waterford']
[' moycullen']
[' sligo']
[' ballinasloe']
[' dublin']
[' co dublin']
[' maynooth']
[' ashford']
[' swords']
[' limerick']
[' athlone']
[' offaly']
[' gorey']
[' dublin 18']
[' castleknock dublin 15']
[' carlow']
[' wilton cork']
[' westport']
[' maxwell road']
[' bray']
[' co kildare']
[' dublin 5']
[' dublin 12']
[' monkstown']
[' pouladuff road']
[' swords']
[' offaly']
[' station road']
[' co cork']
[' dublin']
[' county monaghan']
[' clonmel']
[' dooradoyle road']
[' golden']
[' cork']
[' charleville']
[' dublin']
[' lehenaghbeg']
[' tralee']
[' dublin 12']
[' co. cork']
[' kildare']
[' dublin 24']
[' wexford']
[' farnham']
[' limeri

In [38]:
print(len(addresses_list))

10000


In [39]:
# Print out geocoded addresses to check that geocoding is working before commiting to running on all values
modified_addresses = []
error_addresses = []
geolocator = Nominatim(user_agent = 'property_analysis')

for i in range(len(addresses_list)):
    # print(address[0] + "," + address[1])
    address_string = addresses_list[i][0]
    # print(address_string)
    try:
        location = geolocator.geocode(address_string + ", Ireland")
        modified_addresses.append(location.address)
    except Exception:
        print(f"Geolocator couldn't get the address for address at position {i}")
        modified_addresses.append("Error")
        error_addresses.append(address_string)
    time.sleep(1)

Geolocator couldn't get the address for address at position 48
Geolocator couldn't get the address for address at position 77
Geolocator couldn't get the address for address at position 139
Geolocator couldn't get the address for address at position 188
Geolocator couldn't get the address for address at position 460
Geolocator couldn't get the address for address at position 521
Geolocator couldn't get the address for address at position 700
Geolocator couldn't get the address for address at position 724
Geolocator couldn't get the address for address at position 754
Geolocator couldn't get the address for address at position 770
Geolocator couldn't get the address for address at position 781
Geolocator couldn't get the address for address at position 890
Geolocator couldn't get the address for address at position 937
Geolocator couldn't get the address for address at position 963
Geolocator couldn't get the address for address at position 1109
Geolocator couldn't get the address for a

Geolocator couldn't get the address for address at position 7622
Geolocator couldn't get the address for address at position 7683
Geolocator couldn't get the address for address at position 7774
Geolocator couldn't get the address for address at position 7788
Geolocator couldn't get the address for address at position 7867
Geolocator couldn't get the address for address at position 7919
Geolocator couldn't get the address for address at position 7979
Geolocator couldn't get the address for address at position 8068
Geolocator couldn't get the address for address at position 8149
Geolocator couldn't get the address for address at position 8251
Geolocator couldn't get the address for address at position 8264
Geolocator couldn't get the address for address at position 8326
Geolocator couldn't get the address for address at position 8334
Geolocator couldn't get the address for address at position 8402
Geolocator couldn't get the address for address at position 8433
Geolocator couldn't get t

In [40]:
len(error_addresses)

166

The output of 166 demonstrates that the number of addresses that couldn't be geolocated was a very small portion of the sample. However, the addresses that were successfully geolocated still had very inconsistent outputs with varying levels of detail provided in the address, potentially limiting the effectiveness of using the addresses directly in any predictive modelling.

In [41]:
modified_addresses

['Malahide, Fingal, County Dublin, Leinster, K36 HN84, Éire / Ireland',
 'Parnell Street, Rotunda B ED, Dublin, Dublin 1, Leinster, D01 R243, Éire / Ireland',
 'Ballinamore, Ballinamore Municipal District, County Leitrim, Connacht, Éire / Ireland',
 'Murrough Avenue, Murrough, Cathair na Gaillimhe, County Galway, Connacht, Éire / Ireland',
 'Newbridge, The Municipal District of Kildare — Newbridge, County Kildare, Leinster, W12 AD93, Éire / Ireland',
 'Hansfield, Blanchardstown-Blakestown ED, Fingal, County Dublin, Leinster, Éire / Ireland',
 'Moyvane, Listowel Municipal District, County Kerry, Munster, Éire / Ireland',
 'Cork, County Cork, Munster, T12 T997, Éire / Ireland',
 'Ballyhaunis, Claremorris-Swinford Municipal District, County Mayo, Connacht, F35 NF82, Éire / Ireland',
 'Caherdavin, The Metropolitan District of Limerick City, Limerick, County Limerick, Munster, V94TN5P, Éire / Ireland',
 'Cathair na Gaillimhe, County Galway, Connacht, Éire / Ireland',
 'County Roscommon, Con

In [42]:
error_addresses

[' dunlaoire',
 ' city west dublin',
 ' beaumount woods',
 ' the gallops leopardstown dublin 18',
 ' castaheaney',
 ' ballisodare town centre',
 ' drumcondar',
 ' main st ballycotton',
 ' ashourne',
 ' dualla road cashel',
 ' crandonagh',
 ' county tippeary',
 ' dublin6',
 ' knockenduff',
 ' maree',
 ' crosslanes',
 ' bonniconlon',
 ' martiemo gardens west',
 ' churchbay rd',
 ' and 44/45  southern road',
 ' gretystones',
 ' santry demense',
 ' 35a  35b  40  41 shannon grove',
 ' scallagheen',
 ' maree road',
 ' meellick',
 ' ballycullen  dubin 24',
 ' city west',
 ' dromahaire',
 ' garraneagh',
 ' asbourne',
 ' aikens village',
 ' fourmilehouse',
 ' ballinahowen',
 ' dublin7',
 ' lecamy',
 ' woodford wheaton hall dublin road',
 ' dublin16',
 ' royal canal park  ratoath rd',
 ' drynam hall kinsealy dublin',
 ' thonlagee',
 ' co longfors',
 ' poppinjtree',
 ' casleknock dublin 15',
 ' rochfordbridge',
 ' gslway',
 ' dublin12',
 ' lehenaghbeg',
 ' bretonfield rd',
 ' knock na carra',
 ' 

In [43]:
rppr_df

,Date,Address,Postal_Code,County,Price,Not_Full_Market_Price,VAT_Exclusive,Description_of_Property,Property_Size_Description
0,2016-01-03,"80 THE WATER RILL, WATERSIDE SWORDS RD, MALAHI...",NaN,Dublin,260000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
1,2013-03-15,"apt 40, 109 parnell street",Dublin 1,Dublin,81000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
2,2011-01-14,"Cornabrone, Ballinamore",NaN,Leitrim,155000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
3,2016-10-27,"46 TI CHORMAIC, GLEANN NA RI, MURROUGH AVE",NaN,Galway,60000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
4,2014-08-22,"29 Barrettstown Meadows, Roseberry, NEWBRIDGE",NaN,Kildare,277533.0,No,Yes,New Dwelling house /Apartment,greater than 125 sq metres
...,...,...,...,...,...,...,...,...,...
9995,2021-10-25,"27 AN SRUTHAN, TURLOUGH ROAD, CASTLEBAR",NaN,Mayo,72000.0,Yes,No,Second-Hand Dwelling house /Apartment,NaN
9996,2021-12-16,"1 AN CUAN, GOLF LINKS ROAD, BALLINAMORE",NaN,Leitrim,112000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
9997,2016-08-16,"6 TYRONE MEWS, TYRONE RD, LISMORE LAWN",NaN,Waterford,60000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
9998,2021-11-15,"BALLYSHONAKIN, EFFIN, KILMALLOCK",NaN,Limerick,80000.0,No,No,Second-Hand Dwelling house /Apartment,NaN


In [46]:
# Nested loop used to add error addresses back into address list in their original format to remove
# 'Error' entries

i = 0
j = 0

while i < len(modified_addresses) and j < len(error_addresses):
    if modified_addresses[i] == "Error":
        modified_addresses[i] = error_addresses[j]
        j+=1
    i+=1

In [47]:
modified_addresses[48]

' dunlaoire'

In [49]:
rppr_df['Address'] = modified_addresses
rppr_df

,Date,Address,Postal_Code,County,Price,Not_Full_Market_Price,VAT_Exclusive,Description_of_Property,Property_Size_Description
0,2016-01-03,"Malahide, Fingal, County Dublin, Leinster, K36...",NaN,Dublin,260000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
1,2013-03-15,"Parnell Street, Rotunda B ED, Dublin, Dublin 1...",Dublin 1,Dublin,81000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
2,2011-01-14,"Ballinamore, Ballinamore Municipal District, C...",NaN,Leitrim,155000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
3,2016-10-27,"Murrough Avenue, Murrough, Cathair na Gaillimh...",NaN,Galway,60000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
4,2014-08-22,"Newbridge, The Municipal District of Kildare —...",NaN,Kildare,277533.0,No,Yes,New Dwelling house /Apartment,greater than 125 sq metres
...,...,...,...,...,...,...,...,...,...
9995,2021-10-25,"Castlebar, Castlebar Municipal District, Count...",NaN,Mayo,72000.0,Yes,No,Second-Hand Dwelling house /Apartment,NaN
9996,2021-12-16,"Ballinamore, Ballinamore Municipal District, C...",NaN,Leitrim,112000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
9997,2016-08-16,"Finisterre Lawn, Landsend, Abbeyside, Dungarva...",NaN,Waterford,60000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
9998,2021-11-15,"Kilmallock, The Municipal District of Cappamor...",NaN,Limerick,80000.0,No,No,Second-Hand Dwelling house /Apartment,NaN


## Modifiying Post Codes, Date, VAT and Size Description

In [51]:
rppr_df['Postal_Code']

0            NaN
1       Dublin 1
2            NaN
3            NaN
4            NaN
          ...   
9995         NaN
9996         NaN
9997         NaN
9998         NaN
9999         NaN
Name: Postal_Code, Length: 10000, dtype: category
Categories (22, object): ['Dublin 1', 'Dublin 10', 'Dublin 11', 'Dublin 12', ..., 'Dublin 6w', 'Dublin 7', 'Dublin 8', 'Dublin 9']

In [52]:
type(rppr_df['Postal_Code'][0])

float

In [53]:
postal_code_bool = pd.isnull(rppr_df['Postal_Code'])
postal_code_df = pd.DataFrame(rppr_df[postal_code_bool])
postal_code_df

,Date,Address,Postal_Code,County,Price,Not_Full_Market_Price,VAT_Exclusive,Description_of_Property,Property_Size_Description
0,2016-01-03,"Malahide, Fingal, County Dublin, Leinster, K36...",NaN,Dublin,260000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
2,2011-01-14,"Ballinamore, Ballinamore Municipal District, C...",NaN,Leitrim,155000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
3,2016-10-27,"Murrough Avenue, Murrough, Cathair na Gaillimh...",NaN,Galway,60000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
4,2014-08-22,"Newbridge, The Municipal District of Kildare —...",NaN,Kildare,277533.0,No,Yes,New Dwelling house /Apartment,greater than 125 sq metres
6,2013-12-20,"Moyvane, Listowel Municipal District, County K...",NaN,Kerry,120000.0,No,Yes,New Dwelling house /Apartment,greater than 125 sq metres
...,...,...,...,...,...,...,...,...,...
9995,2021-10-25,"Castlebar, Castlebar Municipal District, Count...",NaN,Mayo,72000.0,Yes,No,Second-Hand Dwelling house /Apartment,NaN
9996,2021-12-16,"Ballinamore, Ballinamore Municipal District, C...",NaN,Leitrim,112000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
9997,2016-08-16,"Finisterre Lawn, Landsend, Abbeyside, Dungarva...",NaN,Waterford,60000.0,No,No,Second-Hand Dwelling house /Apartment,NaN
9998,2021-11-15,"Kilmallock, The Municipal District of Cappamor...",NaN,Limerick,80000.0,No,No,Second-Hand Dwelling house /Apartment,NaN


In [54]:
rppr_df.drop('Property_Size_Description', axis=1, inplace=True)

In [55]:
# Extract year and month data from the date feature and insert into the dataframe

year = rppr_df['Date'].dt.year
month = rppr_df['Date'].dt.month
rppr_df.insert(loc=1, column='Year_of_Sale', value=year)
rppr_df.insert(loc=2, column='Month_of_Sale', value=month)

# Drop the date feature as no longer necessary with year and month features
rppr_df.drop('Date', axis=1, inplace=True)

# Update new features to be categorical data types
rppr_df['Year_of_Sale'] = rppr_df['Year_of_Sale'].astype('category')
rppr_df['Month_of_Sale'] = rppr_df['Month_of_Sale'].astype('category')

In [56]:
# Loop over the dataframe itself, adding VAT to each of the

for idx, row in rppr_df.iterrows():
    if row['VAT_Exclusive'] == 'Yes':
        rppr_df.at[idx, 'Price'] = row['Price'] * 1.135

In [57]:
rppr_df.drop('VAT_Exclusive', axis=1, inplace=True)

### Extra feature for major train routes added to dataset

In [58]:
train_bools = []
train_bools_yes_counties = ['Dublin', 'Galway', 'Wexford', 'Mayo', 'Kerry', 'Sligo', 'Cork', 'Limerick', 'Waterford']

for county in rppr_df['County']:
    if county in train_bools_yes_counties:
        train_bools.append('Yes')
    else:
        train_bools.append('No')

In [59]:
rppr_df.insert(4, 'Major_Train_Route', train_bools)
rppr_df

,Year_of_Sale,Month_of_Sale,Address,Postal_Code,Major_Train_Route,County,Price,Not_Full_Market_Price,Description_of_Property
0,2016,1,"Malahide, Fingal, County Dublin, Leinster, K36...",NaN,Yes,Dublin,260000.000,No,Second-Hand Dwelling house /Apartment
1,2013,3,"Parnell Street, Rotunda B ED, Dublin, Dublin 1...",Dublin 1,Yes,Dublin,81000.000,No,Second-Hand Dwelling house /Apartment
2,2011,1,"Ballinamore, Ballinamore Municipal District, C...",NaN,No,Leitrim,155000.000,No,Second-Hand Dwelling house /Apartment
3,2016,10,"Murrough Avenue, Murrough, Cathair na Gaillimh...",NaN,Yes,Galway,60000.000,No,Second-Hand Dwelling house /Apartment
4,2014,8,"Newbridge, The Municipal District of Kildare —...",NaN,No,Kildare,314999.955,No,New Dwelling house /Apartment
...,...,...,...,...,...,...,...,...,...
9995,2021,10,"Castlebar, Castlebar Municipal District, Count...",NaN,Yes,Mayo,72000.000,Yes,Second-Hand Dwelling house /Apartment
9996,2021,12,"Ballinamore, Ballinamore Municipal District, C...",NaN,No,Leitrim,112000.000,No,Second-Hand Dwelling house /Apartment
9997,2016,8,"Finisterre Lawn, Landsend, Abbeyside, Dungarva...",NaN,Yes,Waterford,60000.000,No,Second-Hand Dwelling house /Apartment
9998,2021,11,"Kilmallock, The Municipal District of Cappamor...",NaN,Yes,Limerick,80000.000,No,Second-Hand Dwelling house /Apartment


In [60]:
# As an aside, pandas seemed to return an error on swapping in new potential values for categorical datatypes
# based on some of the inherent properties of categorical data. Seemingly, this albeit messy solution of creating
# a list of values independent of pandas or a dataframe and then adding values to this list by index worked for
# creating a sequence of values to append to the dataframe in place of the post code information but this was
# admittedly not the ideal solution most likely

postal_codes = []
for i in range(rppr_df.shape[0]):
    if isinstance(rppr_df['Postal_Code'][i], float):
        postal_codes.append(rppr_df['County'][i])
    else:
        postal_codes.append(rppr_df['Postal_Code'][i])
postal_codes

['Dublin',
 'Dublin 1',
 'Leitrim',
 'Galway',
 'Kildare',
 'Dublin 15',
 'Kerry',
 'Cork',
 'Mayo',
 'Limerick',
 'Galway',
 'Roscommon',
 'Roscommon',
 'Dublin 18',
 'Dublin 7',
 'Dublin 5',
 'Cork',
 'Galway',
 'Dublin 12',
 'Donegal',
 'Galway',
 'Dublin 17',
 'Galway',
 'Sligo',
 'Kildare',
 'Kerry',
 'Dublin 22',
 'Cork',
 'Donegal',
 'Dublin 11',
 'Meath',
 'Cork',
 'Offaly',
 'Kilkenny',
 'Dublin 13',
 'Wicklow',
 'Dublin',
 'Clare',
 'Dublin',
 'Dublin 15',
 'Donegal',
 'Dublin 3',
 'Cork',
 'Waterford',
 'Donegal',
 'Dublin 18',
 'Louth',
 'Meath',
 'Dublin',
 'Dublin',
 'Dublin 11',
 'Dublin 14',
 'Dublin 4',
 'Waterford',
 'Dublin 6',
 'Waterford',
 'Cork',
 'Kilkenny',
 'Wicklow',
 'Waterford',
 'Dublin',
 'Dublin 5',
 'Tipperary',
 'Louth',
 'Limerick',
 'Dublin 1',
 'Dublin',
 'Limerick',
 'Wexford',
 'Laois',
 'Kildare',
 'Cork',
 'Roscommon',
 'Cavan',
 'Dublin 18',
 'Cork',
 'Cork',
 'Dublin 15',
 'Limerick',
 'Dublin 3',
 'Dublin',
 'Kerry',
 'Meath',
 'Mayo',
 'Wate

In [61]:
# Insert post code or county feature to replace the post code feature

rppr_df.insert(loc=1, column='Post_Code_or_County', value=postal_codes)
rppr_df.drop('Postal_Code', axis=1, inplace=True)
rppr_df

,Year_of_Sale,Post_Code_or_County,Month_of_Sale,Address,Major_Train_Route,County,Price,Not_Full_Market_Price,Description_of_Property
0,2016,Dublin,1,"Malahide, Fingal, County Dublin, Leinster, K36...",Yes,Dublin,260000.000,No,Second-Hand Dwelling house /Apartment
1,2013,Dublin 1,3,"Parnell Street, Rotunda B ED, Dublin, Dublin 1...",Yes,Dublin,81000.000,No,Second-Hand Dwelling house /Apartment
2,2011,Leitrim,1,"Ballinamore, Ballinamore Municipal District, C...",No,Leitrim,155000.000,No,Second-Hand Dwelling house /Apartment
3,2016,Galway,10,"Murrough Avenue, Murrough, Cathair na Gaillimh...",Yes,Galway,60000.000,No,Second-Hand Dwelling house /Apartment
4,2014,Kildare,8,"Newbridge, The Municipal District of Kildare —...",No,Kildare,314999.955,No,New Dwelling house /Apartment
...,...,...,...,...,...,...,...,...,...
9995,2021,Mayo,10,"Castlebar, Castlebar Municipal District, Count...",Yes,Mayo,72000.000,Yes,Second-Hand Dwelling house /Apartment
9996,2021,Leitrim,12,"Ballinamore, Ballinamore Municipal District, C...",No,Leitrim,112000.000,No,Second-Hand Dwelling house /Apartment
9997,2016,Waterford,8,"Finisterre Lawn, Landsend, Abbeyside, Dungarva...",Yes,Waterford,60000.000,No,Second-Hand Dwelling house /Apartment
9998,2021,Limerick,11,"Kilmallock, The Municipal District of Cappamor...",Yes,Limerick,80000.000,No,Second-Hand Dwelling house /Apartment


### Save dataframe to csv to prevent geolocating again

In [ ]:
rppr_df.to_csv('ppr-14332591-cleaning1')

## Assess clamping outcome for dataset

In [3]:
rppr_df2 = pd.read_csv('ppr-14332591-cleaning1.csv', keep_default_na = True, delimiter=',', skipinitialspace = True)

In [4]:
rppr_df2

,Unnamed: 0,Year_of_Sale,Post_Code_or_County,Month_of_Sale,Address,Major_Train_Route,County,Price,Not_Full_Market_Price,Description_of_Property
0,0,2016,Dublin,1,"Malahide, Fingal, County Dublin, Leinster, K36...",Yes,Dublin,260000.000,No,Second-Hand Dwelling house /Apartment
1,1,2013,Dublin 1,3,"Parnell Street, Rotunda B ED, Dublin, Dublin 1...",Yes,Dublin,81000.000,No,Second-Hand Dwelling house /Apartment
2,2,2011,Leitrim,1,"Ballinamore, Ballinamore Municipal District, C...",No,Leitrim,155000.000,No,Second-Hand Dwelling house /Apartment
3,3,2016,Galway,10,"Murrough Avenue, Murrough, Cathair na Gaillimh...",Yes,Galway,60000.000,No,Second-Hand Dwelling house /Apartment
4,4,2014,Kildare,8,"Newbridge, The Municipal District of Kildare —...",No,Kildare,314999.955,No,New Dwelling house /Apartment
...,...,...,...,...,...,...,...,...,...,...
9995,9995,2021,Mayo,10,"Castlebar, Castlebar Municipal District, Count...",Yes,Mayo,72000.000,Yes,Second-Hand Dwelling house /Apartment
9996,9996,2021,Leitrim,12,"Ballinamore, Ballinamore Municipal District, C...",No,Leitrim,112000.000,No,Second-Hand Dwelling house /Apartment
9997,9997,2016,Waterford,8,"Finisterre Lawn, Landsend, Abbeyside, Dungarva...",Yes,Waterford,60000.000,No,Second-Hand Dwelling house /Apartment
9998,9998,2021,Limerick,11,"Kilmallock, The Municipal District of Cappamor...",Yes,Limerick,80000.000,No,Second-Hand Dwelling house /Apartment


In [7]:
rppr_df2.drop(rppr_df2.columns[[0]], axis=1, inplace=True)
rppr_df2

,Year_of_Sale,Post_Code_or_County,Month_of_Sale,Address,Major_Train_Route,County,Price,Not_Full_Market_Price,Description_of_Property
0,2016,Dublin,1,"Malahide, Fingal, County Dublin, Leinster, K36...",Yes,Dublin,260000.000,No,Second-Hand Dwelling house /Apartment
1,2013,Dublin 1,3,"Parnell Street, Rotunda B ED, Dublin, Dublin 1...",Yes,Dublin,81000.000,No,Second-Hand Dwelling house /Apartment
2,2011,Leitrim,1,"Ballinamore, Ballinamore Municipal District, C...",No,Leitrim,155000.000,No,Second-Hand Dwelling house /Apartment
3,2016,Galway,10,"Murrough Avenue, Murrough, Cathair na Gaillimh...",Yes,Galway,60000.000,No,Second-Hand Dwelling house /Apartment
4,2014,Kildare,8,"Newbridge, The Municipal District of Kildare —...",No,Kildare,314999.955,No,New Dwelling house /Apartment
...,...,...,...,...,...,...,...,...,...
9995,2021,Mayo,10,"Castlebar, Castlebar Municipal District, Count...",Yes,Mayo,72000.000,Yes,Second-Hand Dwelling house /Apartment
9996,2021,Leitrim,12,"Ballinamore, Ballinamore Municipal District, C...",No,Leitrim,112000.000,No,Second-Hand Dwelling house /Apartment
9997,2016,Waterford,8,"Finisterre Lawn, Landsend, Abbeyside, Dungarva...",Yes,Waterford,60000.000,No,Second-Hand Dwelling house /Apartment
9998,2021,Limerick,11,"Kilmallock, The Municipal District of Cappamor...",Yes,Limerick,80000.000,No,Second-Hand Dwelling house /Apartment


In [9]:
rppr_df2.describe().T

,count,mean,std,min,25%,50%,75%,max
Year_of_Sale,10000.0,2016.551500,3.136962,2010.0,2014.0,2017.0,2019.0,2.022000e+03
Month_of_Sale,10000.0,6.882200,3.507271,1.0,4.0,7.0,10.0,1.200000e+01
Price,10000.0,259725.890737,336004.422183,5280.0,120000.0,205000.0,320000.0,1.851000e+07


In [10]:
prices = rppr_df2['Price']

In [12]:
# Check how many prices would be affected by clamping

price_count = 0
for price in prices:
    if price > (320000 + (1.5 * 200000)):
        print(price)
        price_count += 1
print(price_count)

656149.0
649999.745
1300000.0
3500000.0
851251.135
890000.0
645000.0
760000.0
625000.0
635000.0
625000.0
950000.0
1386652.2
685000.0
2500000.0
1315000.0
905000.0
685124.59
865000.0
650000.0
930700.0
620000.56
705000.0
825000.0
750000.0
1225000.0
640000.0
800000.0
680000.0
925000.03
1100000.0
18510000.006750003
750000.0
645000.0
687500.0
810000.0
1900000.0
890000.0
870000.0
704999.99495
740000.0
864999.39
629998.775
675000.0
754000.0
745000.0
635000.0
900000.0
765000.0
729237.5
640000.395
1395000.0
1675000.0
710000.0
875000.0
975000.0
855000.0
700000.0
675900.0
785000.0
627349.96875
1039999.98925
645000.0
2000000.0
1238196.79915
792500.0
760000.0
645000.0
710000.0
3025000.0
685000.0
924998.895
682000.0
822000.0
930000.0
776000.0
735000.0
650000.0
634999.9936
1418750.0
1100000.0
745000.0
700000.0
750000.0
660000.0
815000.0
1100000.0
735000.0
909999.87
660000.0
1200000.0
638000.0
759999.9952
870000.0
1260000.0
840000.0
765000.0
625000.0
1300000.0
630000.0
1100000.0
727500.0
774999.9963
63

In [13]:
rppr_df2['Post_Code_or_County'].unique()

array(['Dublin', 'Dublin 1', 'Leitrim', 'Galway', 'Kildare', 'Dublin 15',
       'Kerry', 'Cork', 'Mayo', 'Limerick', 'Roscommon', 'Dublin 18',
       'Dublin 7', 'Dublin 5', 'Dublin 12', 'Donegal', 'Dublin 17',
       'Sligo', 'Dublin 22', 'Dublin 11', 'Meath', 'Offaly', 'Kilkenny',
       'Dublin 13', 'Wicklow', 'Clare', 'Dublin 3', 'Waterford', 'Louth',
       'Dublin 14', 'Dublin 4', 'Dublin 6', 'Tipperary', 'Wexford',
       'Laois', 'Cavan', 'Dublin 16', 'Monaghan', 'Dublin 9', 'Westmeath',
       'Carlow', 'Dublin 24', 'Longford', 'Dublin 8', 'Dublin 10',
       'Dublin 6w', 'Dublin 2', 'Dublin 20'], dtype=object)

In [14]:
rppr_df2.dtypes

Year_of_Sale                 int64
Post_Code_or_County         object
Month_of_Sale                int64
Address                     object
Major_Train_Route           object
County                      object
Price                      float64
Not_Full_Market_Price       object
Description_of_Property     object
dtype: object

In [15]:
rppr_df2['Post_Code_or_County'] = rppr_df2['Post_Code_or_County'].astype('category')
rppr_df2['Address'] = rppr_df2['Address'].astype('category')
rppr_df2['Major_Train_Route'] = rppr_df2['Major_Train_Route'].astype('category')
rppr_df2['County'] = rppr_df2['County'].astype('category')
rppr_df2['Not_Full_Market_Price'] = rppr_df2['Not_Full_Market_Price'].astype('category')
rppr_df2['Description_of_Property'] = rppr_df2['Description_of_Property'].astype('category')

In [16]:
rppr_df2.dtypes

Year_of_Sale                  int64
Post_Code_or_County        category
Month_of_Sale                 int64
Address                    category
Major_Train_Route          category
County                     category
Price                       float64
Not_Full_Market_Price      category
Description_of_Property    category
dtype: object

In [17]:
rppr_df2

,Year_of_Sale,Post_Code_or_County,Month_of_Sale,Address,Major_Train_Route,County,Price,Not_Full_Market_Price,Description_of_Property
0,2016,Dublin,1,"Malahide, Fingal, County Dublin, Leinster, K36...",Yes,Dublin,260000.000,No,Second-Hand Dwelling house /Apartment
1,2013,Dublin 1,3,"Parnell Street, Rotunda B ED, Dublin, Dublin 1...",Yes,Dublin,81000.000,No,Second-Hand Dwelling house /Apartment
2,2011,Leitrim,1,"Ballinamore, Ballinamore Municipal District, C...",No,Leitrim,155000.000,No,Second-Hand Dwelling house /Apartment
3,2016,Galway,10,"Murrough Avenue, Murrough, Cathair na Gaillimh...",Yes,Galway,60000.000,No,Second-Hand Dwelling house /Apartment
4,2014,Kildare,8,"Newbridge, The Municipal District of Kildare —...",No,Kildare,314999.955,No,New Dwelling house /Apartment
...,...,...,...,...,...,...,...,...,...
9995,2021,Mayo,10,"Castlebar, Castlebar Municipal District, Count...",Yes,Mayo,72000.000,Yes,Second-Hand Dwelling house /Apartment
9996,2021,Leitrim,12,"Ballinamore, Ballinamore Municipal District, C...",No,Leitrim,112000.000,No,Second-Hand Dwelling house /Apartment
9997,2016,Waterford,8,"Finisterre Lawn, Landsend, Abbeyside, Dungarva...",Yes,Waterford,60000.000,No,Second-Hand Dwelling house /Apartment
9998,2021,Limerick,11,"Kilmallock, The Municipal District of Cappamor...",Yes,Limerick,80000.000,No,Second-Hand Dwelling house /Apartment


In [18]:
rppr_df2.to_csv('ppr-14332591-clean.csv')